<a href="https://colab.research.google.com/github/joseluiz1990/joseluiz1990/blob/main/ED_Atv03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 03

## Qual API foi utilizada e o motivo da escolha?

URL: https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/aplicacao#!/recursos/CotacaoDolarPeriodo

## Qual dado ou dados foram extraídos?

Foram extraídos os seguintes dados da API

* cotação de compra
* cotação de venda
* data e hora da cotação (depois foi removida a hora e deixado apenas a data)

## Como a função lambda foi aplicada e por que?

Foram aplicadas duas funções lambdas

1) uma para calcular o ágio absoluto, ou seja, a diferença entre a cotação de venda e de compra na função de armazenamento do dicionario

2) outra para calcular o ágio percentual, ou seja, o percentual que o ágio absoluto representa em relação a cotação de compra e venda na função para exibir a pesquisa

In [2]:
import requests

# Função de requisição à API do Banco Central para obter cotações diárias do dollar em um período de tempo
def pesquisar_dollar(**data): # usando kwargs para passar as datas inicial e final
  data_inicial = data['date_start']
  data_final = data['date_end']
  url = f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='"+data_inicial+"'&@dataFinalCotacao='"+data_final+"'&$top=20000&$format=json&$select=cotacaoCompra,cotacaoVenda,dataHoraCotacao"
  resposta = requests.get(url)
  if resposta.status_code == 200:
    dados = resposta.json()
    return dados
  else:
    return {'Erro': 'Não foi possivel obter dados'}

# Função para processar os dados e armazená-los em um dicionário.
def armazenar_dic(dados):
  cotacao_dollar={}
  for i in range(len(dados['value'])):
    compra=(dados['value'][i]['cotacaoCompra'])
    venda=(dados['value'][i]['cotacaoVenda'])
    data=(dados['value'][i]['dataHoraCotacao'])
    data=data[:10] # pega somente o dia e remove a hora

    # usado função lambda dentro da função armazenar dicionario para calcular o valor percentual de ágio entre compra e venda do dollar
    diferenca_percentual = lambda sell, buy: ((sell-buy) / buy) * 100
    valor = round(diferenca_percentual(venda,compra),4) # arredonda o valor para 4 casas decimais

    cotacao_dollar[data] = {'cotacao_compra':compra, 'cotacao_venda':venda, 'diferenca_percent':valor}
  return cotacao_dollar

# Função para exibir as informações formatadas para o usuário.
def exibir_pesquisa(cotacao_dollar):
  print(f'\n---Cotações de Compra e Venda de Dollar diária em um período ---\n')
  print('-' * 55)
  for data,valores in cotacao_dollar.items():
    print(f"Data          : {data}")
    print(f"Cotação Compra: {cotacao_dollar[data]['cotacao_compra']}")
    print(f"Cotação Venda : {cotacao_dollar[data]['cotacao_venda']}")
    print(f"Diferença %   : {cotacao_dollar[data]['diferenca_percent']}")

    # usado função lambda dentro da função exibir pesquisa para calcular o valor absoluto de ágio entre compra e venda do dollar
    diferenca = lambda dif: dif[data]['cotacao_venda'] - dif[data]['cotacao_compra']
    valor = diferenca(cotacao_dollar)
    print(f"Diferença     : {valor:.4f}")
    print('-' * 55)

# Programa Principal
continuar = 's'
while continuar == 's': # enquanto continuar "sim", executa ....
  data_inicial = input('Digite a data inicial no formato mm-dd-aaaa: ')
  data_final = input('Digite a data final no formato mm-dd-aaaa: ')
  dados = pesquisar_dollar(date_start=data_inicial, date_end=data_final) # pesquisa a cotação diária do dollar na API do Banco Central, passando parametros nomeados
  if 'Erro' not in dados: # se não há erro no retorno da pesquisa, então...
    cotacao_dollar = armazenar_dic(dados) # armazena os dados da pesquisa em dicionario de dicionarios, a chave principal é a Data da cotação
    exibir_pesquisa(cotacao_dollar) # exibe a pesquisa
  else: # senão ....
    print(dados['Erro']) # imprime a mensagem de erro da pesquisa
  continuar = input('Deseja continuar? [s/n]')


Digite a data inicial no formato mm-dd-aaaa: 03-05-2025
Digite a data final no formato mm-dd-aaaa: 03-07-2025

---Cotações de Compra e Venda de Dollar diária em um período ---

-------------------------------------------------------
Data          : 2025-03-05
Cotação Compra: 5.7908
Cotação Venda : 5.7914
Diferença %   : 0.0104
Diferença     : 0.0006
-------------------------------------------------------
Data          : 2025-03-06
Cotação Compra: 5.7483
Cotação Venda : 5.7489
Diferença %   : 0.0104
Diferença     : 0.0006
-------------------------------------------------------
Data          : 2025-03-07
Cotação Compra: 5.7682
Cotação Venda : 5.7688
Diferença %   : 0.0104
Diferença     : 0.0006
-------------------------------------------------------
Deseja continuar? [s/n]n
